# Table of Contents

*  [Rooftop Solution Introduction](#modelintro)
   *  [Model Target](#modeltarget)
   *  [In This Analysis We Will](#analysissteps) 
   *  [In this Notebook We Create](#outputlist)
   
   <br>

*  [Explore the Input File](#exploreinput)
   *  [Import Packages and Load the Evaluation Dataset](#import)
   *  [Address Errors & Geocoding Hit Rate](#hitrate)
   *  [Addresses Distribution by States](#statesdist)
   *  [Address Distribution by Regions](#regionsdist)
   *  [Snapdate Distribution](#snapdatedist)
   *  [Other Attribute Distribution (year_built)](#year-built-distribution)

   
   <br>

*  [Evaluate the Solution](#evaluatesolution)
   * [Score Distribution National and Regional](#scoredist)
   * [How to Update Regional and National Target Variables](#updatetarget)
   * [Overall Claim Rates Stability](#over-all-claimrates)
   * [National Gains Charts](#claimratesgainschart)


   
   <br>

*  [Pricing Model Evaluation](#pricingmodelevaluation)
   * [Gains Chart by old rooftop score](#oldrooftopscore)
   
   <br>

*  [Multiple Snap Dates for Same Address](#multiple)
   * [Gains Chart by Snapdate Year](#snapdatesgainschart)
   * [Difference in Score Distribution between Snapdates](#scoredistbeweensnapdates)
   
   <br>

*  [Appendix](#appendix)
   * [Customer Facing Attributes Fill Rate](#attributesfillrate)
   * [Hit Rate Summary](#hitratefull)

   
   

# Rooftop Solution lntroduction: <a id='modelintro'></a>

LexisNexis Rooftop is a series of loss propensity models that produce a score to predict the liklihood of a future weather-related claim. It is built with two types of data: geospatially aggregated attributes and property characteristics from public records. The geospatially aggregations are on claims and publicly-available weather attributes. The solution consists of 6 different models across 4 regions across the entire United States (including AK and HI). 

* <b>Central Zone 1:</b> TX, OK, AR
* <b>Central Zone 2:</b> IL, CO, NE, MO, IA, KS, NM, MT, SD, ND, WY
* <b>South Region:</b> AL, FL, GA, KY, LA, MS, NC, SC, TN, VA
* <b>Northeast Zone 1:</b> CT, DC, DE, IN, MA, MD, ME, MI, NH
* <b>Northeast Zone 2:</b> NJ, NY, OH, PA, RI, VT, WI, WV
* <b>West Region: </b> CA, AK, AZ, HI, ID, NV, OR, UT, WA

The scores are calibrated between 1 and 100 so that the numerical score represents the same level of risk, regardless of which underlying model produced the result. A score of 1 represents the lowest risk of a future weather-related claim and a score of 100 represents the highest risk of a future weather-related claim. 

 ## Model Target <a id='modeltarget'></a>
 The target for each model includes open and closed claims with a claim amount over $5,000. These include both Catastrophe and Non-Catastrophe losses. The weather perils included in each model target differs based on the region. 

<b>Central and South Region Target</b>
 * Wind, Hail, or Weather-Water peril losses that occur in the 12 months following the date the record was scored (`snapdate`)

<b> Northeast and West Region Target</b>
 * Wind, Hail, Weather-Water, Water, Freeze, and Miscellaneous peril losses that occur in the 18 months following the date the record was scored (`snapdate`)

Using CLUE Commercial & Property contributions, we were able to identify properties with a claim in the 12 -18 months following the snapdate. Claims from Business Owner, Multi-Peril, Commercial Property, and Personal Property policies were considered.

## In This Analysis We Will: <a id='analysissteps'></a>

<b>Explore the input file</b><br>
    Before we begin understanding the model performance, we will first understand the input file's addresses and snap dates (the dates in which the record is to be scored). We will explore whether or not the addresses in the file are valid, scorable locations and which regions those locations cover. We also need to confirm the snap dates are valid. Dates prior to 2017 may have some inconsistencies and dates within the last year will not have had time for claim development. Additionally, we'll check the distribution and fill rate of any other interesting data elements within the file that we may wish to evaluate later on.
<br><br>   
<b>Evaluate the Solution</b><br>
    Once we are comfortable with our input file, we will shift our focus to the actual model score. First we will look at the distribution of the score on the input addresses. Next we will show you how to update the target variable fields (`addr_future_claim_indicator` and `addr_future_claim_amount`) with carrier-sourced claims data. We will then look at the stability of the model target before evaluating the performance of the score on the input sample.  

## In This Notebook We Create: <a id='outputlist'></a>
* [<b>Hit rate charts</b> to see the percentage of properties that can be scored](#hitrate)
* [<b>State distribution chart</b> to understand which model regions are relevant](#statesdist)
* [<b>Region distribution chart</b>](#regionsdist)
* [<b>Snapdate distribution chart</b> to understand if some records should be excluded from the evaluation](#snapdatedist) 
* [<b>Year_built distribution chart</b> as an example to check distribution and fill rate for attributes in the file](#year-built-distribution)
* [<b>Score distribution chart</b> to understand the segmentation and spread of the score](#scoredist)
* [<b>Updated target variables</b> to measure score performance against](#createregionaltarget)
* [<b>Claim rate by year chart</b> to understand if the updated target is stable enough over time to analyze](#over-all-claimrates)
* [<b>National Claim Rate, Claim Severity, and Loss Ratio gains charts</b> to show model performance](#claimratesgainschart)
* [Analysis highlighting significant score differences across snap dates for the same property](#multiple)

<br><br>
# Explore the Input File <a id='exploreinput'></a>
<br><br>

## Import Packages <a id='import'></a>
First we will import the packages needed for the jupyter notebook to run. The yml file can be used to create a conda enviroment which contains all the needed packages.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use("default")
warnings.filterwarnings("ignore")

##  Load the evalution dataset <a id='load'></a>
LexisNexis returns a file with the Rooftop score and attributes appended to the inpute file. This file contains all customer fields in addition to the Rooftop score and over 40 attributes that can be created. Because the input fields are returned back to the customer in this file, it should be possible to join this back to any internal datasets you may have. The most important field appended to the input dataset is `new_score` - the rooftop score field. 

In order to proceed with the evaluation, we must inport this evaluation file. We assume the evaluation file and evaluation notebook are in the same directory on your machine. 

In [ ]:
# import customer input file with rooftop score file,its pipe separated csv file
df = pd.read_csv("./Out_scoreoutput.csv", sep="|")


## Geocoding Hit Rate <a id='hitrate'></a>
Every address must be precisely geocoded. There is a `geo_match` field that is returned as a part of address standardization with a numerical value of 0, 1, 4, or 5. A value of 0 means we can identify the coordinates to a precise point within the parcel. We require geo_match to equal "0" in order to ensure that our geospatially aggregated attributes are correct for the input address. If an address does not have a value of "0" in the field, we may still be able to return some attributes that don't require geospatial aggregation despite returning 999 for the score. 

In [ ]:
# geocode quality check up,hit means good geocode, no hit means not able to geocode.
print("generating geocoding hit rate")
df["geo_match_ind"] = df.geo_match == 0
report = df.geo_match_ind.value_counts().reset_index()
report.columns = ["geo_match", "count"]
report["rate"] = report["count"] / report["count"].sum()
report["geo_match"] = report["geo_match"].map({True: "hit", False: "no hit"})
report["rate"] = report["rate"].apply("{:.2%}".format)
fig, ax = plt.subplots(figsize=(5, 2))
ax.axis("off")
c = report.shape[1]
ax.table(
    cellText=np.vstack([report.columns, report.values]),
    cellColours=[["lightgrey"] * c] + [["white"] * c] * report.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Geocoding Hit Rate (Total: {})".format(df.shape[0]), loc="left")
plt.tight_layout()

## Address Standardization Error Rate <a id='errorrate'></a>
In addition to precise geocoding, every address must have a valid address. Properties with a precise geo_match typically have a valid address, but there will be some properties that have a valid address despite the lack of precision on the coordinates. To understand which properties did not pass address standardization, we must take a look at the `err_stat` column that is returned in the file. If the value in this column starts with `E`, there is an error that prohibited LexisNexis from standardizing the address. All properties that fail address standardization will be returned with a rooftop score of 999. These addresses will not have any geospatial or property characteristic attributes returned. 

In [ ]:
df["err_stat_ind"] = df.err_stat.str[0] == "E"
report = df.err_stat_ind.value_counts().reset_index()
report.columns = ["err_stat", "count"]
report["rate"] = report["count"] / report["count"].sum()
report["err_stat"] = report["err_stat"].map({True: "error", False: "no error"})
report["rate"] = report["rate"].apply("{:.2%}".format)
fig, ax = plt.subplots(figsize=(5, 2))
ax.axis("off")
c = report.shape[1]
ax.table(
    cellText=np.vstack([report.columns, report.values]),
    cellColours=[["lightgray"] * c] + [["none"] * c] * report.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Error Rate (Total: {})".format(df.shape[0]), loc="left")
plt.tight_layout()

## Hit Rate Summary
Combining the geocoding hit rate and address standardization error rate, we are able to get a full understanding of the number of records that were scorable, the number of records with at least some attributes, and the number of records that did not pass either geocoding or address standardization. 

In [ ]:
##################### overall summary###############
print("generating overall summary...")

types = ["Total # of Records", "Address Found", "Score Returned"]
x = []
x.append(df.shape[0])  # total # of records
x.append(df[df.err_stat.str[0] != "E"].shape[0])  # address found
x.append(df[df.new_score < 101].shape[0])  # scores returned

report = pd.DataFrame({"Types": types, "# of Records": x})
report["Fill Rate"] = report["# of Records"] / report["# of Records"][0]

report["Fill Rate"] = report["Fill Rate"].apply("{:.2%}".format)
fig, ax = plt.subplots()  # figsize=(5,max(report.shape[0]/3.5, 2))
ax.axis("off")
c = report.shape[1]
ax.table(
    cellText=np.vstack([report.columns, report.values]),
    cellColours=[["lightgray"] * c] + [["none"] * c] * report.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Overall Summary", loc="left")
plt.tight_layout()

## Address Distribution by State<a id='statesdist'></a>
Because the model is region based, its important to know which regions are represented in the input file. Later in this analysis we will update the model target based on the regions. When evaluating regional performance, it will be important to consider volume of properties in that region. If there are not enough properties in a particular region, the performance may not be indicitive of what you would expect with a statistically significant sample. In this case, we recommend paying closer attention to the national performance.

In [ ]:
print("generating distribution by state")
zone1_st = ["OK", "TX", "AR"]
zone2_st = ["CO", "NE", "MO", "MN", "IA", "KS", "NM", "MT", "SD", "ND", "WY", "IL"]
south_st = ["GA", "SC", "FL", "NC", "MS", "KY", "TN", "VA", "LA", "AL"]
west_st = ["AK", "AZ", "CA", "HI", "ID", "NV", "OR", "UT", "WA"]
ne_st = [
    "CT",
    "DC",
    "DE",
    "IN",
    "MA",
    "MD",
    "ME",
    "MI",
    "NH",
    "NJ",
    "NY",
    "OH",
    "PA",
    "RI",
    "VT",
    "WI",
    "WV",
]
model_st = zone1_st + zone2_st + south_st + ne_st + west_st
report = df[df.state.isin(model_st)].state.value_counts().reset_index()
report.columns = ["statecode", "count"]
fig, ax = plt.subplots(
    figsize=(max(report.shape[0] / 3, 5), max(report.shape[0] / 6, 3))
)
sns.barplot(x="statecode", y="count", data=report, ax=ax, color="gray")
ax.set_title("Distribution of Addresses", loc="left")
ax.grid()
plt.tight_layout()
report = report.sort_values(by=["count"], ascending=False)
report["percentage"] = report["count"] / report["count"].sum()
report["percentage"] = report["percentage"].apply("{:.2%}".format)
fig, ax = plt.subplots(figsize=(7, max(report.shape[0] / 3, 2)))
ax.axis("off")
c = report.shape[1]
ax.table(
    cellText=np.vstack([report.columns, report.values]),
    cellColours=[["lightgray"] * c] + [["none"] * c] * report.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Distribution of Addresses (Scorable States)", loc="left")
plt.tight_layout()

## Address Distribution by Region<a id='regionsdist'></a>
In order to understand if there is sufficient volume to analyze the Rooftop solution regionally, we will take a look at the distribution of addresses by region with both a numerical chart and graph. 

In [ ]:
print("generating distribution by regions")

def assign_zone(state):
    if state in zone1_st:
        return "Central"
    elif state in zone2_st:
        return "Central"
    elif state in south_st:
        return "South"
    elif state in west_st:
        return "West"
    elif state in ne_st:
        return "NorthEast"
    else:
        return "Unknown"

df["regions"] = df["state"].apply(assign_zone)

report = df["regions"].value_counts().reset_index()
report.columns = ["regions", "count"]
fig, ax = plt.subplots(
    figsize=(max(report.shape[0] / 3, 5), max(report.shape[0] / 6, 3))
)
sns.barplot(x="regions", y="count", data=report, ax=ax, color="gray")
ax.set_title("Distribution of Addresses by regions", loc="left")
ax.grid()
plt.tight_layout()
report = report.sort_values(by=["count"], ascending=False)
report["percentage"] = report["count"] / report["count"].sum()
report["percentage"] = report["percentage"].apply("{:.2%}".format)
fig, ax = plt.subplots(figsize=(7, max(report.shape[0] / 3, 2)))
ax.axis("off")
c = report.shape[1]
ax.table(
    cellText=np.vstack([report.columns, report.values]),
    cellColours=[["lightgray"] * c] + [["none"] * c] * report.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Distribution of Addresses (Scorable States)", loc="left")
plt.tight_layout()

## Snapdate distribution <a id='snapdatedist'></a>

Each record is scored as of a particular `snapdate`. Because claims history begins to thin as we consider dates further in the past, we recommend only evaluating properties with a snapdate after January 1, 2017. We also tend not to evaluate records with a snapdate within 18 months of today's date. This ensures that claims have had time to develop so that we can test if the score was preditive. This distribution will help determine if any filters need to be applied before evaluating the solution later in this analysis. 

In [ ]:
df = df[(df.new_score < 101) & (df.state.isin(model_st))].reset_index(
    drop=True
)
print("generating snapdate distribution")
fig, ax1 = plt.subplots(figsize=(6, 3.5))
df["snapyear"] = pd.to_datetime(df.snapdate, errors="coerce").dt.year
report = df.snapyear.value_counts()
sns.barplot(
    x=report.index, y=report.values, ax=ax1, color="gray"
)
ax1.set_title("Distribution of Snapdate", loc="left")
plt.tight_layout()

## Other Attribute Distribution <a id='year-built-distribution'></a>

You may decide that there are additional fields you'd like to evaluate as a part of the Rooftop solution. A common field that is evaluated is `year_built`. Below is an example of how you can create distributions for other fields that are of interest. 

In [ ]:
df = df[(df.new_score < 101) & (df.state.isin(model_st))].reset_index(
    drop=True
)
print("generating yearbuilt distribution")
fig, ax2 = plt.subplots(figsize=(6, 3.5))
df["buildyear"] = df.pc_yearbuilt.dropna().astype(int)
df["buildyear"] = np.maximum(df.buildyear, 1800)
df[df.buildyear > 1800]["buildyear"].astype(float).hist(ax=ax2, bins=100, color="gray")
ax2.set_title("Distribution of Yearbuilt (>1800)", loc="left")
plt.tight_layout()

<br><br>
# Evaluate the Solution <a id='evaluatesolution'></a>
<br><br>

## Score Distribution <a id='scoredist'></a>
The score predicts the liklihood that a property will have a future weather-related claim in the 12 - 18 months following the snapdate. The first thing we want to test when evaluating the score is whether or not properties are segmented across the entire range of potential scores. By looking at the score distribution we can determine if scores are bunched around a single score or spread out across the range. We can also understand if the properties submitted for this test are skewed toward high or low future weather-related claim risk. The Rooftop scores increase as future weather-related risk increases, meaning a score of 100 represents a property with the highest risk of a future weather-related claim. 

In [ ]:
print("generating score distribution")
target = "addr_future_claim_indicator"
fig, ax = plt.subplots(figsize=(10, 7))
df["new_score"].astype(float).hist(ax=ax, bins=50, edgecolor="white", color="gray")
ax.set_xlabel("score")
ax.set_title("Scores, Nationwide", loc="left")
plt.tight_layout()
temp1 = df[df.state.isin(zone1_st + zone2_st)]
temp2 = df[df.state.isin(south_st)]
temp3 = df[df.state.isin(ne_st)]
temp4 = df[df.state.isin(west_st)]

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 4.5))

if temp1.shape[0] != 0:
    temp1["new_score"].astype(float).hist(
        ax=ax1, bins=50, edgecolor="white", color="gray"
    )
    ax1.set_xlabel("score")
    ax1.set_title("Scores, Central Region", loc="left")

if temp2.shape[0] != 0:
    temp2["new_score"].astype(float).hist(
        ax=ax2, bins=50, edgecolor="white", color="gray"
    )
    ax2.set_xlabel("score")
    ax2.set_title("Scores, South Region", loc="left")

if temp3.shape[0] != 0:
    temp3["new_score"].astype(float).hist(
        ax=ax3, bins=50, edgecolor="white", color="gray"
    )
    ax3.set_xlabel("score")
    ax3.set_title("Scores, Northeast Region", loc="left")

if temp4.shape[0] != 0:
    temp4["new_score"].astype(float).hist(
        ax=ax4, bins=50, edgecolor="white", color="gray"
    )
    ax4.set_xlabel("score")
    ax4.set_title("Scores, West Region", loc="left")

plt.tight_layout()

## Updating Target Variable <a id='updatetarget'></a>
The precise target for this model varies depending on the region the property is in. In this section we will define the target more precisely and use the included loss columns in the file to instruct you on how to update the `addr_future_claim_indicator` and `addr_future_claim_amt` fields. Though our example won't actually change the target variables, it can be used as a template for a more customized analysis. The included target variable field use CLUE Commercial & Property contributions to identify pclaims in the 12 -18 months following the snapdate. Claims from Business Owner, Multi-Peril, Commercial Property, and Personal Property policies were considered. 

Once we've updated the target variables, we'll be able to evaluate the score's ability to predict future weather-related claims. In order to do this, we must first update the target variable field regionally and then combine the regional targets to create a "national" target. 

### Generating Regional Targets <a id='createregionaltarget'></a>

For the Central and South regional models, the target includes ***claims over $5,000 related to Wind, Hail, Weather Water perils 12 months after the snapdate***.

For the Northeast and West region, the model target includes ***claims over $5,000 related to Wind, Hail, Weather Water, Water, and Freeze perils within 18 months after the snapdate***.

First we will update the target variables for the Central and South Regions before updating the target variables for the Northeast and West regional models.



#### Central & South Targets
The states in the Central region are as follows: ['OK','TX','AR','CO','NE','MO','MN','IA','KS','NM','MT','SD','ND','WY','IL']

The states in the South region are as follows: ['GA', 'SC', 'FL', 'NC', 'MS', 'KY', 'TN', 'VA', 'LA', 'AL']
   

#### Northeast & West Targets

  The states in the West region are as follows = ['AK', 'AZ', 'CA', 'HI', 'ID', 'NV', 'OR', 'UT', 'WA']

  The states in the Northeast region are as follows   = ['CT', 'DC', 'DE', 'IN', 'MA', 'MD', 'ME', 'MI', 'NH', 'NJ', 'NY', 'OH', 'PA', 'RI', 'VT', 'WI', 'WV']

In [ ]:

temp_ne_west = df[df.state.isin(ne_st + west_st)]

# hail, wind, and water,and other related loss columns.
values_to_match = ['WEATHER','HAIL','WIND','FREEZE','WATER']
loss_columns=['AMOUNT_PAID']
#create future loss amount


#create future loss amount
temp_ne_west['addr_future_claim_amt_ne_west'] = temp_ne_west[temp_ne_west['summary_cause_of_loss'].isin(values_to_match)][loss_columns].apply(lambda x: x[x > 5000].sum(), axis=1)

# create```addr_future_claim_indicator```

temp_ne_west.loc[temp_ne_west['addr_future_claim_amt_ne_west'] > 0, 'addr_future_claim_indicator_ne_west'] = 1


# merge back to the LexisNexis return file
df_merge_ne_west = pd.merge(df, temp_ne_west[['uid','addr_future_claim_amt_ne_west','addr_future_claim_indicator_ne_west']], how='left', on='uid')

### Generating National Target <a id='createnationaltarget'></a>
Once the regional targets have been updated individually, we merge the records back together for our "national" target. 

In [ ]:
df_merge = df_merge_ne_west

# ```addr_future_claim_indicator```

df_merge["addr_future_claim_amt"] = df_merge[
    ["addr_future_claim_amt_ne_west"]
].apply(lambda x: x[x > 0].sum(), axis=1)


# create```addr_future_claim_indicator```

df_merge.loc[df_merge["addr_future_claim_amt"] > 0, "addr_future_claim_indicator"] = 1

## Overall Claim Rates <a id='over-all-claimrates'></a>

Now that the target has been updated, we can take a look at the volatility of the target to ensure it is consistent enough to be analyzed further. Claim rates vary from year to year so the goal isn't necessarily to see perfectly stable performance, but to assess if the claim rate is sufficient enough over time to include all valid snap dates. We sometimes see more recent snap dates with less time for claim development with a much different claim rate than the other properties in the file. 

In [ ]:
##################### claim rate time series
print("generating claim rate time series")
df_merge["snapdate"] = pd.to_datetime(df_merge.snapdate, errors="coerce")
df_merge["date"] = df_merge.snapdate.dt.to_period("M")

df_merge["addr_future_claim_indicator"] = df_merge["addr_future_claim_indicator"].fillna(0)

claim_rates = (df_merge.groupby("date").addr_future_claim_indicator.sum() /
              df_merge.groupby("date").addr_future_claim_indicator.count()).reset_index()

claim_rates.columns = ["date", "claim rate"]

fig, ax = plt.subplots()
claim_rates.plot.line(x="date", y="claim rate", ax=ax, color="gray")
ax.set_title("Claim Rate Volatility", loc="left")
ax.set_xlabel("snapdate")
ax.set_ylabel("claim rate")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.grid()

## National Gains Charts
With our [updated target variables](#updatetarget), we will evaluate the predictive power of the model overall. Though the model is meant to predict claim frequency or rate, we provide gains charts for claim severity as well. If Premium is provided, we can consider the performance of Rooftop above and beyond what is already being charged by assessing Loss Ratio in our gains charts. A successful gains chart will have a good spread of properties across the bins with claim rate, severity, or loss ratio increasing as the bin values increase. <br> 

In order to visualize these gains charts we created a function called  `create_gains` capable of plotting claim rate, claim severity, and loss ratio gains charts.

* `df` is the dataframe name which is used to generate gains chart.
* `title` takes a string as the plot title name.
* `x_label` takes a string as the x axis label name.
* `y_label` takes a string as the left side y axis name (eg.exposure).
* `y_label2` takes a string as the right side y axis name.
* `claim_rates=True` indicates to the function to plot claim rates gains chart.
* `loss_ratio=True` indicates to the function to plot loss ratio gains chart.
* `claim_severity=True` indicates to function to plot claim severity gains chart.
* `cuts` controls the number of bins in the gains chart. In this notebook, we used 5, 10, 20 bins.
* `target` column is the field with the model target. The default is `addr_future_claim_indicator`, which was generated earlier in this notebook.
* `future_loss` column is the field with the total loss dollars associated with target claims. The default is `addr_future_claim_amt` column which was also generated earlier in this notebook.
* `earned_premium` points to the premium column name as the denominator for the loss_ratio equation.

In [ ]:
# define a plot gains chart function for plotting purposes.
def create_gains(
    ax=None,
    df=None,
    title=None,
    x_label=None,
    y_label=None,
    y_label2=None,
    claim_rates=None,
    loss_ratio=None,
    claim_severity=None,
    cuts=None,
    target="addr_future_claim_indicator",
    future_loss="addr_future_claim_amt",
    earned_premium=None,
):
    ax1 = ax
    fig, ax1 = plt.subplots(figsize=(6, 4))
    ax2 = ax1.twinx()
    df["exposure"] = 1
    df["bins"] = pd.cut(df["new_score"], cuts)
    if claim_rates is not None:
        gains = df.groupby("bins").agg({"exposure": "sum", target: "sum"}).reset_index()
        gains["claim_rate"] = gains[target] / gains["exposure"]
        avg_claim_rate = round(df[target].sum() / df.shape[0] * 100, 2)
        gains["claim_rate"].plot(
            kind="line", c="black", lw=2, ls="--", marker="o", ax=ax2
        )  # c='red'
        gains = gains.reset_index()
        ax1.set_xlabel("Average Claim Rate: {}%".format(avg_claim_rate))
        if y_label2 is not None:
            ax2.set_ylabel(y_label2)
            ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1))

    if claim_severity is not None:
        gains = (
            df.groupby("bins")
            .agg({future_loss: "sum", "exposure": "sum", target: "sum"})
            .reset_index()
        )
        gains["claim_severity"] = gains[future_loss] / gains[target]
        gains["claim_severity"].plot(
            kind="line", c="black", lw=2, ls="--", marker="o", ax=ax2
        )  # c='red'
        if y_label2 is not None:
            ax2.set_ylabel(y_label2)
            ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter("${x:,.2f}"))
        if x_label is not None:
            ax1.set_xlabel(x_label)

    if loss_ratio is not None:
        gains = (
            df.groupby("bins")
            .agg({future_loss: "mean", "exposure": "sum", earned_premium: "mean"})
            .reset_index()
        )
        gains["loss_ratio"] = gains[future_loss] / gains[earned_premium]
        gains["loss_ratio"].plot(
            kind="line", c="black", lw=2, ls="--", marker="o", ax=ax2
        )  # c='red'
        if y_label2 is not None:
            ax2.set_ylabel(y_label2)
            ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter("${x:,.2f}"))
        if x_label is not None:
            ax1.set_xlabel(x_label)

    gains["exposure"].plot(
        kind="bar", ax=ax1, color="gray", width=0.9
    )  # color='skyblue'
    ax1.set_xticklabels(gains["bins"], rotation=80)
    ax1.set_ylim([0, gains["exposure"].max() * 2.5])
    ax1.tick_params(labelbottom=True)
    ax1.xaxis.grid()
    ax2.grid()
    ax1.set_xticklabels(gains["bins"], rotation=80)
    if title is not None:
        ax1.set_title(title, loc="left")
    if y_label is not None:
        ax1.set_ylabel(y_label)
    plt.tight_layout()

### National Claim Rate Gains Chart <a id='claimratesgainschart'></a>
The Rooftop models are all trained to segment properties on their liklihood of a future weather-related claim. A successful rooftop test will show the claim rate increasing as the rooftop score also increases. In this section we will create several gains charts to test this. Each gains chart will have Rooftop score bins along the x-axis. The scores will be grouped in 5 bins, 10 bins, and 20 bins. There will be two y-axes: One for the bars that show the percentage of properties in each bin and another for the line that shows claim rate for the properties in each bin. The claims in each bin are based on the [updated target variables](#updatetarget) created in the earlier section. 

In [ ]:
create_gains(
    df=df_merge,
    title="national_claim_rate",
    x_label="Rooftop_Score",
    y_label="exposure",
    y_label2="claim_rate",
    claim_rates=True,
    cuts=np.arange(0, 120, 20),
    target="addr_future_claim_indicator",
)

create_gains(
    df=df_merge,
    title="national_claim_rate",
    x_label="Rooftop_Score",
    y_label="exposure",
    y_label2="claim_rate",
    claim_rates=True,
    cuts=np.arange(0, 110, 10),
    target="addr_future_claim_indicator",
)

create_gains(
    df=df_merge,
    title="national_claim_rate",
    x_label="Rooftop_Score",
    y_label="exposure",
    y_label2="claim_rate",
    claim_rates=True,
    cuts=np.arange(0, 105, 5),
    target="addr_future_claim_indicator",
)

### National Claim Severity Gains Chart <a id='claimseveritygainschart'></a>
Though the Rooftop models are not trained on loss amount or severity, it's still important to check if there is an unexpected relationship between the model score and loss amounts. To explore the claim severity we will create a gains chart with the Rooftop score separated into 10 bins along the x-axis. The gains chart will have two y-axes: One for the bars that  show the percentage of properties in each bin and another for the line that shows the average claim severity for the properties in each bin. If there is a relationship between claim severity and Rooftop score, we will see severity increase as the score increases. The claims in each bin are based on the [updated target variables](#updatetarget) created in the earlier section. 

In [ ]:
# use claim amount to calculate claim severity
# 10 bins
create_gains(
    df=df_merge,
    title="national_claim_severity",
    x_label="Rooftop_Score",
    y_label="exposure",
    y_label2="claim_severity",
    claim_severity=True,
    cuts=np.arange(0, 110, 10),
    target="addr_future_claim_indicator",
    future_loss="addr_future_claim_amt",
)

#  Multiple snap dates for same address <a id='multiple'></a>
The input file contained multiple snap dates for the same address. To understand the model performance over time we will take a look at the claim rate gains chart by snap date year. We will also identify the properties that had a significant change in Rooftop score (a difference of 10 or more points) for potential further investigation. 


## Claim Rate Gains Charts by Snap Date Year  <a id='snapdatesgainschart'></a>
In this section of the analysis we will filter each of our [National Claim Rate Gains Charts](#claimratesgainschart) by the snap date year. Here we want to see that the performance of the Rooftop score is stable over time (especially as we get closer to current day). For each snap date year, a successful test will show the claim rate increasing as the rooftop score also increases. Each gains chart will have Rooftop score grouped into 10 bins along the x-axis. There will be two y-axes: One for the bars that show the percentage of properties in each bin and another for the line that shows claim rate for the properties in each bin. The claims in each bin are based on the [updated target variables](#updatetarget) created in the earlier section. 

In [ ]:
# performance by year
Years = sorted(df.snapyear.unique())
for year in Years:
    temp = df_merge[df_merge.snapyear == year]
    if temp[target].sum() != 0:
        create_gains(
            df=temp,
            title="claim_rate_by_year_{}".format(year),
            x_label="Rooftop_Score",
            y_label="exposure",
            y_label2="claim_rate",
            claim_rates=True,
            cuts=np.arange(0, 110, 10),
            target="addr_future_claim_indicator",
            future_loss="addr_future_claim_amt",
        )


## Difference in Score Distribution <a id='scoredistbeweensnapdates'></a> 

Further investigation may be desired for specific properties whose Rooftop score significantly changed between snap dates. In this section we will identify the properties with multiple snap dates and measure the difference in score between them. We will then identify the properties with a significant change in score (defined as a change of 10 points or more). To visualize this we create a distribtion graph of the score differences. 

* Based on below analysis, across multiple snapdate for same addresses

   * The largest score drop or raise between snap dates is 85.00
   
   * The percentage of the score drop or raise (more than 10 points) between snap dates is 14.46%

In [ ]:
# get the min max score difference group by address tract key
df = df[df.new_score < 101]
score_diff = (
    df.groupby(["addr_tract_key"])
    .agg({"new_score": ["min", "max", lambda x: max(x) - min(x)]})
    .reset_index()
)

score_diff.columns = ["addr_tract_key", "min_score", "max_score", "score_diff"]

# plot score difference for all the addresses
fig, ax = plt.subplots(figsize=(8, 5))
score_diff["score_diff"].astype(float).hist(
    ax=ax, bins=50, edgecolor="white", color="gray"
)
ax.set_xlabel("score difference")
ax.set_title("Scores Difference across multiple snapdate for same address", loc="left")
plt.tight_layout()


# bin the score different, and plot by score difference bins
score_diff["score_cuts"] = pd.cut(score_diff.score_diff, [-np.inf, 1, 10, 15, 50, 80])
score_diff["exposure"] = 1
tab = score_diff.groupby("score_cuts").agg({"exposure": "sum"}).reset_index()
fig, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
tab["exposure"].plot(kind="bar", ax=ax1, color="gray")  # color='skyblue'
ax1.set_xticklabels(tab.score_cuts, rotation=60)
ax1.set_ylim([0, tab["exposure"].max() * 2.5])
ax1.set_ylabel("exposure")
ax1.set_title("Score difference with multiple snapdate for same address", loc="left")
ax1.xaxis.grid()
for ax in (ax1, ax2):
    ax.tick_params(right=False, labelright=False)
ax1.set_ylabel("exposure")

ax2.grid()
plt.tight_layout()

diff_percentage_greater_than_0 = (score_diff["score_diff"] > 0).mean() * 100
diff_percentage_greater_than_10 = (score_diff["score_diff"] >= 10).mean() * 100
print(
    "The largest score drop or raise between snap dates is {:.2f}".format(
        score_diff.score_diff.max()
    )
)
print(
    "The percentage of score drop or raise (more than 0 points) between snap dates is {:.2f}%".format(
        diff_percentage_greater_than_0
    )
)
print(
    "The percentage of score drop or raise (more than 10 points) between snap dates is {:.2f}%".format(
        diff_percentage_greater_than_10
    )
)

#  Pricing Model Evaluation <a id='pricingmodelevaluation'></a>

space holder


## Gains chart by old rooftop score <a id='oldrooftopscore'></a>
In this section of the analysis we will see the national gains chart by the old rooftop score as well.

In [ ]:
# define a plot gains chart function for plotting purposes.
def create_gains(
    ax=None,
    df=None,
    title=None,
    x_label=None,
    y_label=None,
    y_label2=None,
    claim_rates=None,
    cuts=None,
    target="addr_future_claim_indicator",
):
    ax1 = ax
    fig, ax1 = plt.subplots(figsize=(6, 4))
    ax2 = ax1.twinx()
    df["exposure"] = 1
    df["bins"] = pd.cut(df["score_RTGEOV01"], cuts)
    if claim_rates is not None:
        gains = df.groupby("bins").agg({"exposure": "sum", target: "sum"}).reset_index()
        gains["claim_rate"] = gains[target] / gains["exposure"]
        avg_claim_rate = round(df[target].sum() / df.shape[0] * 100, 2)
        gains["claim_rate"].plot(
            kind="line", c="black", lw=2, ls="--", marker="o", ax=ax2
        )  # c='red'
        gains = gains.reset_index()
        ax1.set_xlabel("Average Claim Rate: {}%".format(avg_claim_rate))
        if y_label2 is not None:
            ax2.set_ylabel(y_label2)
            ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1))

    gains["exposure"].plot(
        kind="bar", ax=ax1, color="gray", width=0.9
    )  # color='skyblue'
    ax1.set_xticklabels(gains["bins"], rotation=80)
    ax1.set_ylim([0, gains["exposure"].max() * 2.5])
    ax1.tick_params(labelbottom=True)
    ax1.xaxis.grid()
    ax2.grid()
    ax1.set_xticklabels(gains["bins"], rotation=80)
    if title is not None:
        ax1.set_title(title, loc="left")
    if y_label is not None:
        ax1.set_ylabel(y_label)
    plt.tight_layout()


In [ ]:
create_gains(
    df=df_merge,
    title="national_claim_rate",
    x_label="Old_Rooftop_Score",
    y_label="exposure",
    y_label2="claim_rate",
    claim_rates=True,
    cuts=np.arange(0, 120, 20),
    target="addr_future_claim_indicator",
)

create_gains(
    df=df_merge,
    title="national_claim_rate",
    x_label="Old_Rooftop_Score",
    y_label="exposure",
    y_label2="claim_rate",
    claim_rates=True,
    cuts=np.arange(0, 110, 10),
    target="addr_future_claim_indicator",
)

create_gains(
    df=df_merge,
    title="national_claim_rate",
    x_label="Old_Rooftop_Score",
    y_label="exposure",
    y_label2="claim_rate",
    claim_rates=True,
    cuts=np.arange(0, 105, 5),
    target="addr_future_claim_indicator",
)

# Appendix <a id='appendix'></a>
## Customer Facing Attributes Fill Rate  <a id='attributesfillrate'></a>
In addition to the Rooftop score, the return file comes with additional attributes. These attribute may be useful inputs to a pricing or underwriting process. In this section we will determine the fill rate of the customer-facing attributes. 

In [ ]:
customer_layout = {
    "pc_roofing_material": str,
    "pc_yearbuilt": int,
    "pc_approx_roofareasf": float,
    "pc_roof_type": str,
    "years_since_known_weather_loss_5k": str,
    "nearby_claim_cnt_hail_1y": float,
    "nearby_claim_cnt_wind_1y": float,
    "nearby_claim_cnt_wwater_1y": float,
    "hail_claim_cluster_10y": float,
    "auto_claim_cluster_3y": float,
    "region_hail_frequency": float,
    "region_wind_frequency": float,
    "cnt_hail_1_year": float,
    "cnt_hail_10_year": float,
    "cnt_wind_1_year": float,
    "cnt_wind_10_year": float,
    "cnt_wind_180_days": float,
    "landcover": str,
    "recent_hail_event_date": int,
    "recent_wind_event_date": int,
    "total_market_value": float,
    "new_score": int,
    "nearby_claim_cnt_water_1y": int,
    "nearby_claim_cnt_freeze_1y": int,
    "water_claim_cluster_10y": float,
    "region_water_frequency": float,
    "daily_avg_temp_fahrenheit": float,
    "distance_greatlake_miles": float,
    "distance_coast_miles": float,
    "area_landform_local_relief": float,
    "area_landform_slope": str,
    "area_landform_profile_type": str,
    "nearby_ice_cnt_1y": int,
    "nearby_ice_cnt_10y": int,
    "nearby_cold_cnt_1y": int,
    "nearby_cold_cnt_10y": int,
    "nearby_winter_cnt_1y": int,
    "nearby_winter_cnt_10y": int,
    "nearby_snow_cnt_1y": int,
    "nearby_snow_cnt_10y": int,
    "wthr_max_temp_summer": float,
    "wthr_max_temp_fall": float,
    "wthr_max_temp_spring": float,
    "wthr_min_temp_summer": float,
    "wthr_avg_temp_1y": float,
    "wthr_avg_sum_temp_1y": float,
    "wthr_avg_prcp_1y": float,
    "wthr_cnt_temp_b_fz_10y": float,
    "wthr_cnt_temp_min_max_fz_1y": float,
}

In [ ]:
# read customer attributes to a new df
df = pd.read_csv("./Out_scoreoutput.csv", sep="|")
df_customer = df[customer_layout.keys()]

In [ ]:
##################### fill rate (customer facing attributes)
fill_counts = []

print("generating attribute fill rate for customer use")

fill_rates = []
for attr in df_customer:
    if attr == "new_score":
        cnt = df_customer[
            (df_customer[attr].isna())
            | (df_customer[attr] < 0)
            | (df_customer[attr] > 100)
        ].shape[0]
        fill_counts.append(df_customer.shape[0] - cnt)
        fill_rates.append(1 - cnt / df_customer.shape[0])
    else:
        if df_customer[attr].dtype.name == "object":
            cnt = df_customer[
                (df_customer[attr].isna())
                | (df_customer[attr].astype(str).isin(["-1.0", "-1"]))
            ].shape[0]
            fill_counts.append(df_customer.shape[0] - cnt)
            fill_rates.append(1 - cnt / df_customer.shape[0])
        else:
            cnt = df_customer[
                (df_customer[attr].isna()) | (df_customer[attr].astype(float) == -1)
            ].shape[0]
            fill_counts.append(df_customer.shape[0] - cnt)
            fill_rates.append(1 - cnt / df_customer.shape[0])

report1 = pd.DataFrame({"attribute": df_customer.columns, "fill rate": fill_rates})
report1 = report1.sort_values(by=["attribute"])
report1["fill rate"] = report1["fill rate"].apply("{:.2%}".format)
fig, ax = plt.subplots(figsize=(7, report1.shape[0] / 3))
ax.axis("off")
c = report1.shape[1]
ax.table(
    cellText=np.vstack([report1.columns, report1.values]),
    cellColours=[["lightgray"] * c] + [["none"] * c] * report1.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Attribute Fill Rate", loc="left")
plt.tight_layout()



## Hit Rate Summary<a id="hitratefull"></a>
Using the returned customer-facing attributes, we can update the Hit Rate Summary to include the number of records with at least one appended attribute. 

In [ ]:
##################### overall summary###############
print("generating overall summary...")

types = ["Total # of Records", "Address Found", "Score Returned", "Attributes Returned"]
x = []
x.append(df.shape[0])  # total # of records
x.append(df[df.err_stat.str[0] != "E"].shape[0])  # address found
x.append(df[df.new_score < 101].shape[0])  # scores returned
x.append(max(fill_counts)) # attributes returned

report = pd.DataFrame({"Types": types, "# of Records": x})
report["Fill Rate"] = report["# of Records"] / report["# of Records"][0]

report["Fill Rate"] = report["Fill Rate"].apply("{:.2%}".format)
fig, ax = plt.subplots()  # figsize=(5,max(report.shape[0]/3.5, 2))
ax.axis("off")
c = report.shape[1]
ax.table(
    cellText=np.vstack([report.columns, report.values]),
    cellColours=[["lightgray"] * c] + [["none"] * c] * report.shape[0],
    bbox=[0, 0, 1, 1],
)
ax.set_title("Overall Summary", loc="left")
plt.tight_layout()